# Data Embedding

In [62]:
%pip install datasets openai pinecone-client cohere

Note: you may need to restart the kernel to use updated packages.


In [63]:
from datasets import load_dataset

data = load_dataset("jamescalam/ai-arxiv-chunked", split="train")
data

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 41584
})

In [64]:
data = data.map(lambda x: {
    "id": f'{x["id"]}-{x["chunk-id"]}',
    "text": x["chunk"],
    "metadata": {
        "title": x["title"],
        "url": x["source"],
        "primary_category": x["primary_category"],
        "published": x["published"],
        "updated": x["updated"],
        "text": x["chunk"],
    }
})
data = data.remove_columns([
    "journal_ref", "primary_category", "published", "updated", "references", "authors", "categories", "comment", "title", "summary", "source", "doi", "chunk-id", "chunk"
])

In [65]:
import os
import openai
import getpass

In [66]:
openai_key = os.getenv("OPENAI_API_KEY", getpass.getpass("Enter your OpenAI API key: "))

openai.api_key = openai_key

In [67]:
from openai import OpenAI

In [68]:
client = OpenAI(
  api_key=openai_key
)

In [69]:
embed_model = "text-embedding-ada-002"

In [77]:
def embed(docs: list[str]) -> list[list[float]]:
    res = client.embeddings.create(input=docs, model=embed_model)
    embeddings = [e.embedding for e in res.data]
    return embeddings

In [78]:
from pinecone import Pinecone, ServerlessSpec
import time

In [79]:
api_key = os.getenv("PINECONE_API_KEY", getpass.getpass("Enter your Pinecone API key: "))

pc = Pinecone(
    api_key=api_key
)

In [80]:
pc.list_indexes().indexes

[{'dimension': 1536,
  'host': 'rerankers-r3rtzs8.svc.apw5-4e34-81fa.pinecone.io',
  'metric': 'dotproduct',
  'name': 'rerankers',
  'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
  'status': {'ready': True, 'state': 'Ready'}}]

In [81]:
index_name = "rerankers"

if not any(d['name'] == index_name for d in pc.list_indexes().indexes):

    pc.create_index(
        name="rerankers",
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-west-2'
        ) 
    ) 

    while not pc.describe_index(index_name).status["ready"]:
        print("Waiting for index to be ready...")
        time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [82]:
from tqdm.auto import tqdm

In [83]:
batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    passed = False
    # find end of batch
    i_end = min(len(data), i+batch_size)
    # create batch
    batch = data[i:i_end]
    embeds = embed(batch["text"])
    to_upsert = list(zip(batch["id"], embeds, batch["metadata"]))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|██████████| 416/416 [11:42<00:00,  1.69s/it]


In [86]:
def get_docs(query: str, top_k: int) -> list[str]:
    # encode query
    xq = embed([query])[0]
    # search pinecone index
    res = index.query(vector=xq, top_k=top_k, include_metadata=True)
    # get doc text
    docs = {x["metadata"]['text']: i for i, x in enumerate(res["matches"])}
    return docs

In [87]:
query = "can you explain why we would want to do rlhf?"
docs = get_docs(query, top_k=25)
print("\n---\n".join(docs.keys()))

whichmodels areprompted toexplain theirreasoningwhen givena complexproblem, inorder toincrease
the likelihood that their ﬁnal answer is correct.
RLHF has emerged as a powerful strategy for ﬁne-tuning Large Language Models, enabling signiﬁcant
improvements in their performance (Christiano et al., 2017). The method, ﬁrst showcased by Stiennon et al.
(2020) in the context of text-summarization tasks, has since been extended to a range of other applications.
In this paradigm, models are ﬁne-tuned based on feedback from human users, thus iteratively aligning the
models’ responses more closely with human expectations and preferences.
Ouyang et al. (2022) demonstrates that a combination of instruction ﬁne-tuning and RLHF can help ﬁx
issues with factuality, toxicity, and helpfulness that cannot be remedied by simply scaling up LLMs. Bai
et al. (2022b) partially automates this ﬁne-tuning-plus-RLHF approach by replacing the human-labeled
ﬁne-tuningdatawiththemodel’sownself-critiquesandrevisions,

# Reranking

In [88]:
import cohere

In [89]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY", getpass.getpass("Enter your Cohere API key: "))

co = cohere.Client(os.environ["COHERE_API_KEY"])

In [90]:
rerank_docs = co.rerank(
    query=query, documents=docs.keys(), top_n=25, model="rerank-english-v2.0"
)

In [92]:
rerank_docs[0].document["text"]

'whichmodels areprompted toexplain theirreasoningwhen givena complexproblem, inorder toincrease\nthe likelihood that their ﬁnal answer is correct.\nRLHF has emerged as a powerful strategy for ﬁne-tuning Large Language Models, enabling signiﬁcant\nimprovements in their performance (Christiano et al., 2017). The method, ﬁrst showcased by Stiennon et al.\n(2020) in the context of text-summarization tasks, has since been extended to a range of other applications.\nIn this paradigm, models are ﬁne-tuned based on feedback from human users, thus iteratively aligning the\nmodels’ responses more closely with human expectations and preferences.\nOuyang et al. (2022) demonstrates that a combination of instruction ﬁne-tuning and RLHF can help ﬁx\nissues with factuality, toxicity, and helpfulness that cannot be remedied by simply scaling up LLMs. Bai\net al. (2022b) partially automates this ﬁne-tuning-plus-RLHF approach by replacing the human-labeled\nﬁne-tuningdatawiththemodel’sownself-critiquesan

In [93]:
[docs[doc.document["text"]] for doc in rerank_docs]


[0,
 21,
 14,
 3,
 12,
 9,
 6,
 8,
 1,
 15,
 7,
 20,
 2,
 17,
 10,
 19,
 16,
 24,
 13,
 23,
 22,
 4,
 18,
 11,
 5]

In [94]:
def compare(query: str, top_k: int, top_n: int):
    # first get vec search results
    docs = get_docs(query, top_k=top_k)
    i2doc = {docs[doc]: doc for doc in docs.keys()}
    # rerank
    rerank_docs = co.rerank(
        query=query, documents=docs.keys(), top_n=top_n, model="rerank-english-v2.0"
    )
    original_docs = []
    reranked_docs = []
    # compare order change
    for i, doc in enumerate(rerank_docs):
        rerank_i = docs[doc.document["text"]]
        print(str(i)+"\t->\t"+str(rerank_i))
        if i != rerank_i:
            reranked_docs.append(f"[{rerank_i}]\n"+doc.document["text"])
            original_docs.append(f"[{i}]\n"+i2doc[i])
    for orig, rerank in zip(original_docs, reranked_docs):
        print("ORIGINAL:\n"+orig+"\n\nRERANKED:\n"+rerank+"\n\n---\n")

In [95]:
compare(query, 25, 3)

0	->	0
1	->	21
2	->	14
ORIGINAL:
[1]
We examine the inﬂuence of the amount of RLHF training for two reasons. First, RLHF [13, 57] is an
increasingly popular technique for reducing harmful behaviors in large language models [3, 21, 52]. Some of
these models are already deployed [52], so we believe the impact of RLHF deserves further scrutiny. Second,
previous work shows that the amount of RLHF training can signiﬁcantly change metrics on a wide range of
personality, political preference, and harm evaluations for a given model size [41]. As a result, it is important
to control for the amount of RLHF training in the analysis of our experiments.
3.2 Experiments
3.2.1 Overview
We test the effect of natural language instructions on two related but distinct moral phenomena: stereotyping
and discrimination. Stereotyping involves the use of generalizations about groups in ways that are often
harmful or undesirable.4To measure stereotyping, we use two well-known stereotyping benchmarks, BBQ
[40] 

In [96]:
compare("what is red teaming?", top_k=25, top_n=3)

0	->	0
1	->	4
2	->	16
ORIGINAL:
[1]
red-teaming expertise valuable for organizations with suf ﬁcient resources. However, it would also be
beneﬁcial to experiment with the formation of a community of AI red teaming professionals that draws
together individuals from different organizations and bac kgrounds, speciﬁcally focused on some subset
of AI (versus AI in general) that is relatively well-deﬁned a nd relevant across multiple organizations.25
A community of red teaming professionals could take actions such as publish best practices, collectively
analyze particular case studies, organize workshops on eme rging issues, or advocate for policies that
would enable red teaming to be more effective.
Doing red teaming in a more collaborative fashion, as a commu nity of focused professionals across
23Red teaming could be aimed at assessing various properties o f AI systems, though we focus on safety and security in this
subsection given the expertise of the authors who contribut ed to it.
24F

In [97]:
pc.delete_index(index_name) #saving resources bc pinecone bills u for active indexes